## Skorch basic classification example
#### from skorch docs/readme - https://github.com/skorch-dev/skorch

In [ ]:
import subprocess

# Installation on Google Colab
try:
    import google.colab
    subprocess.run(['python', '-m', 'pip', 'install', 'skorch' , 'torch'])
except ImportError:
    pass

In [1]:
import numpy as np
from sklearn.datasets import make_classification
import torch
from torch import nn
from skorch import NeuralNetClassifier
from skorch import NeuralNet
from pyperch.utils.decorators import add_to
from skorch.dataset import unpack_data
import copy

In [2]:
X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)
input_dim=20
output_dim=2

class MyModule(nn.Module):
    def __init__(self, num_units=10, nonlin=nn.ReLU()):
        super().__init__()

        self.dense0 = nn.Linear(input_dim, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, num_units)
        self.output = nn.Linear(num_units, output_dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense1(X))
        X = self.softmax(self.output(X))
        return X

    #warning: this is only a POC for replacing the backprop training step
    def run_rhc_single_step(self, net, old_loss, X_train, y_train, **fit_params):
        # Randomly perturb the weights
        for param in net.module_.parameters():
            param.data += torch.randn(param.data.size())

net = NeuralNetClassifier(
    MyModule,
    max_epochs=20,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

@add_to(NeuralNet)
def train_step_single(self, batch, **fit_params):
    self._set_training(True)
    Xi, yi = unpack_data(batch)
    y_pred = self.infer(Xi, **fit_params)
    loss = self.get_loss(y_pred, yi, X=Xi, training=True)
    loss.backward()
    return {
        'loss': loss,
        'y_pred': y_pred,
    }

net.fit(X, y)
#y_proba = net.predict_proba(X)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6772       0.5600        0.6792  0.0191
      2        0.6689       0.5600        0.6727  0.0170
      3        0.6732       0.5850        0.6687  0.0239
      4        0.6549       0.6050        0.6625  0.0208
      5        0.6480       0.5950        0.6531  0.0212
      6        0.6392       0.6500        0.6433  0.0218
      7        0.6338       0.6350        0.6360  0.0197
      8        0.6246       0.6800        0.6273  0.0213
      9        0.6218       0.6900        0.6214  0.0196
     10        0.6138       0.6800        0.6141  0.0219
     11        0.5963       0.6900        0.6105  0.0220
     12        0.5917       0.6950        0.6028  0.0202
     13        0.5832       0.7000        0.5934  0.0171
     14        0.6044       0.7050        0.5917  0.0163
     15        0.5710       0.6950        0.5849  0.0243
     16        0.5852       0.7

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (nonlin): ReLU()
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
    (softmax): Softmax(dim=-1)
  ),
)

## train_step_single override - disable backprop and run single step RO

In [3]:
@add_to(NeuralNet)
def train_step_single(self, batch, **fit_params):
    self._set_training(True)
    Xi, yi = unpack_data(batch)
    y_pred = self.infer(Xi, **fit_params)
    loss = self.get_loss(y_pred, yi, X=Xi, training=True)
    #disable backprop and run custom training step
    #loss.backward()
    self.module_.run_rhc_single_step(self, loss, Xi, yi, **fit_params)
    return {
        'loss': loss,
        'y_pred': y_pred,
    }
    
net.fit(X, y)

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        6.6552       0.5250        7.5281  0.0207
      2        7.7238       0.5450        7.2538  0.0175
      3        7.5357       0.5100        7.8051  0.0188
      4        8.8077       0.3800        9.8843  0.0173
      5        9.2500       0.3600       10.2031  0.0149
      6        9.1963       0.4350        9.0074  0.0191
      7        8.3647       0.4300        9.0872  0.0192
      8        8.7690       0.4450        8.8480  0.0120
      9        8.3299       0.4450        8.8480  0.0160
     10        8.6592       0.4150        9.3263  0.0113
     11        8.4861       0.4800        8.2900  0.0138
     12        8.4581       0.4900        8.1306  0.0132
     13        8.5973       0.4050        9.4857  0.0202
     14        8.7484       0.4150        9.3263  0.0144
     15   

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (nonlin): ReLU()
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
    (softmax): Softmax(dim=-1)
  ),
)

## Using sklearn pipeline with RO

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', net),
])

pipe.fit(X, y)
y_proba = pipe.predict_proba(X)

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.9104       0.5450        7.0622  0.0216
      2        7.4245       0.5400        7.3335  0.0193
      3        7.7064       0.4850        8.1023  0.0202
      4        8.2736       0.4950        8.0509  0.0106
      5        7.7185       0.5000        7.9712  0.0168
      6        8.3648       0.4200        9.2466  0.0138
      7        8.7790       0.4300        8.9799  0.0213
      8        8.5292       0.4450        8.8480  0.0132
      9        8.3698       0.4900        8.1306  0.0184
     10        8.7043       0.4400        8.9277  0.0143
     11        8.2198       0.4550        8.6886  0.0197
     12        7.8749       0.4850        8.0999  0.0210
     13        7.7974       0.5100        7.6660  0.0176
     14        8.2497       0.4950        8.0509  0.0149
     15   

## Using sklearn grid search with RO

In [5]:
from sklearn.model_selection import GridSearchCV

# deactivate skorch-internal train-valid split and verbose logging
net.set_params(train_split=False, verbose=0)
params = {
    'lr': [0.01, 0.02],
    'max_epochs': [10, 20],
    'module__num_units': [10, 20],
}
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy', verbose=2)

gs.fit(X, y)
print("best score: {:.3f}, best params: {}".format(gs.best_score_, gs.best_params_))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END .......lr=0.01, max_epochs=10, module__num_units=10; total time=   0.1s
[CV] END .......lr=0.01, max_epochs=10, module__num_units=10; total time=   0.1s
[CV] END .......lr=0.01, max_epochs=10, module__num_units=10; total time=   0.1s
[CV] END .......lr=0.01, max_epochs=10, module__num_units=20; total time=   0.1s
[CV] END .......lr=0.01, max_epochs=10, module__num_units=20; total time=   0.1s
[CV] END .......lr=0.01, max_epochs=10, module__num_units=20; total time=   0.1s
[CV] END .......lr=0.01, max_epochs=20, module__num_units=10; total time=   0.2s
[CV] END .......lr=0.01, max_epochs=20, module__num_units=10; total time=   0.2s
[CV] END .......lr=0.01, max_epochs=20, module__num_units=10; total time=   0.2s
[CV] END .......lr=0.01, max_epochs=20, module__num_units=20; total time=   0.2s
[CV] END .......lr=0.01, max_epochs=20, module__num_units=20; total time=   0.2s
[CV] END .......lr=0.01, max_epochs=20, module__n